In [2]:
import pandas as pd
import numpy as np
import string
string .punctuation
data = pd.read_excel(r'C:/Users/shiya/OneDrive/Desktop/textsummarizer/datasetnomissingvalues.xlsx')
print(data)

                                                   text
0      If you're a photographer, keep all the necess...
1      See the image for how this drawing develops s...
2      It is possible to become a VFX artist without...
3      The best art investors do their research on t...
4      As you start planning for a project or work, ...
...                                                 ...
2887   If you have consistently low oxygen saturatio...
2888   Breathing in smoke from tobacco products seri...
2889   It helps your body increase its production of...
2890   Exercising regularly boosts your body’s immun...
2891   Layering your clothing helps to hold in your ...

[2892 rows x 1 columns]


In [3]:
import re

def remove_punctuation_and_keep_fullstops(text):
    if isinstance(text, str):
        # Remove all characters except full stops and convert to lowercase
        cleaned_text = re.sub(r'[^\w\s.]', '', text).lower()
        return cleaned_text
    else:
        return str(text)


In [4]:
data['text'] = data['text'].apply(remove_punctuation_and_keep_fullstops)

In [5]:
data

,text
0,if youre a photographer keep all the necessar...
1,see the image for how this drawing develops s...
2,it is possible to become a vfx artist without...
3,the best art investors do their research on t...
4,as you start planning for a project or work y...
...,...
2887,if you have consistently low oxygen saturatio...
2888,breathing in smoke from tobacco products seri...
2889,it helps your body increase its production of...
2890,exercising regularly boosts your bodys immune...


In [6]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize

def tokenize_text(text):
    # Tokenize the text into sentences.
    sentences = sent_tokenize(text)

    # Tokenize each sentence into words.
    tokenized_text = []
    for sentence in sentences:
        words = word_tokenize(sentence)
        tokenized_text.append(words)

    return tokenized_text




In [7]:
data['tokenized_text'] = data['text'].apply(tokenize_text)

In [8]:

topics= pd.read_excel(r'C:\Users\shiya\OneDrive\Desktop\textsummarizer\TopicsGensim.xlsx')
topics

,text
0,"['exercise', 'study', 'found', 'also', 'weight..."
1,"['crush', 'time', 'conversation', 'something',..."
2,"['girl', 'cat', 'dog', 'way', 'dont', 'get', '..."
3,"['feel', 'cry', 'feeling', 'might', 'make', 'e..."
4,"['crush', 'get', 'feeling', 'may', 'contact', ..."
...,...
2887,"['may', 'time', 'friend', 'like', 'try', 'cat'..."
2888,"['crush', 'like', 'dance', 'make', 'go', 'try'..."
2889,"['time', 'get', 'life', 'new', 'may', 'also', ..."
2890,"['hamster', 'like', 'post', 'picture', 'ad', '..."


In [9]:
df=pd.DataFrame()
df['tokenized_text']=data['tokenized_text']
df['topics']=topics
df

,tokenized_text,topics
0,"[[if, youre, a, photographer, keep, all, the, ...","['exercise', 'study', 'found', 'also', 'weight..."
1,"[[see, the, image, for, how, this, drawing, de...","['crush', 'time', 'conversation', 'something',..."
2,"[[it, is, possible, to, become, a, vfx, artist...","['girl', 'cat', 'dog', 'way', 'dont', 'get', '..."
3,"[[the, best, art, investors, do, their, resear...","['feel', 'cry', 'feeling', 'might', 'make', 'e..."
4,"[[as, you, start, planning, for, a, project, o...","['crush', 'get', 'feeling', 'may', 'contact', ..."
...,...,...
2887,"[[if, you, have, consistently, low, oxygen, sa...","['may', 'time', 'friend', 'like', 'try', 'cat'..."
2888,"[[breathing, in, smoke, from, tobacco, product...","['crush', 'like', 'dance', 'make', 'go', 'try'..."
2889,"[[it, helps, your, body, increase, its, produc...","['time', 'get', 'life', 'new', 'may', 'also', ..."
2890,"[[exercising, regularly, boosts, your, bodys, ...","['hamster', 'like', 'post', 'picture', 'ad', '..."


In [10]:
from sklearn.feature_extraction.text import CountVectorizer
import networkx as nx

In [11]:
import pandas as pd
import numpy as np
from scipy.sparse import lil_matrix

def build_co_occurrence_matrix(text, topics):
    # Create a vocabulary
    vocabulary = set(word for sentence in text for word in sentence)
    vocabulary_mapping = {word: idx for idx, word in enumerate(vocabulary)}
    
    # Initialize a co-occurrence matrix
    co_occurrence_matrix = lil_matrix((len(vocabulary), len(vocabulary)), dtype=np.int32)
    
    # Assign extra weights to sentences containing topics
    for doc_index, doc in enumerate(text):
        word_count = {}  # Dictionary for words for each document
        for word in doc:
            if word not in word_count:
                word_count[word] = 0
            word_count[word] += 1

        # Assign a higher weight for documents containing topics
        if topics and any(topic in doc for topic in topics):
            for word1 in doc:
                for word2 in doc:
                    # Increase the co-occurrence count
                    co_occurrence_matrix[vocabulary_mapping[word1], vocabulary_mapping[word2]] += 1

    return co_occurrence_matrix



In [12]:
def summarize_text(text,topics):
    co_occurrence_matrix = build_co_occurrence_matrix(text, topics)
    G = nx.Graph(co_occurrence_matrix)
    textrank_scores = nx.pagerank(G, alpha=0.85)
    num_top_sentences = 6
    sentences = [', '.join(sublist) for sublist in text]
    sorted_sentences = sorted(sentences, key=lambda sentence: textrank_scores.get(sentence, 0), reverse=True)
    summary = sorted_sentences[:num_top_sentences]
    return summary


In [13]:
all_summaries = []
for row in df.itertuples():
    if not isinstance(row.tokenized_text, list):
        continue
    summary = summarize_text(row.tokenized_text, row.topics)
    all_summaries.append(summary)
    

In [14]:
summ=pd.DataFrame()
summ['Summary']=all_summaries

In [15]:
summ

,Summary
0,"[if, youre, a, photographer, keep, all, the, n..."
1,"[see, the, image, for, how, this, drawing, dev..."
2,"[it, is, possible, to, become, a, vfx, artist,..."
3,"[the, best, art, investors, do, their, researc..."
4,"[as, you, start, planning, for, a, project, or..."
...,...
2887,"[if, you, have, consistently, low, oxygen, sat..."
2888,"[breathing, in, smoke, from, tobacco, products..."
2889,"[it, helps, your, body, increase, its, product..."
2890,"[exercising, regularly, boosts, your, bodys, i..."


In [16]:
all_summaries

[['if, youre, a, photographer, keep, all, the, necessary, lens, cords, and, batteries, in, the, same, quadrant, of, your, home, or, studio, .',
  'paints, should, be, kept, with, brushes, cleaner, and, canvas, print, supplies, should, be, by, the, ink, etc, .',
  'make, broader, groups, and, areas, for, your, supplies, to, make, finding, them, easier, limiting, your, search, to, a, much, smaller, area, .',
  'some, ideas, include, essential, supplies, area, the, things, you, use, every, day, .',
  'inspiration, and, reference, area, .',
  'dedicated, work, area, .'],
 ['see, the, image, for, how, this, drawing, develops, stepbystep, .',
  'however, there, is, an, important, detail, the, following, drawings, are, to, examine, it, and, then, to, create, something, unique, .',
  'use, the, lines, to, create, the, image, shape, and, sections, .',
  'fill, appeared, sections, with, different, patterns, ornaments, .',
  'add, text, if, needed, for, example, neopoprealism, is, 25, add, a, col